### <font color='orange'>Привет Артем ! Переделал2, немного заморочился , но так будет лучше  </font>
### <font color='orange'>Привет Артем ! Переделал </font>
Привет, меня зовут Артем Хуршудов. Сегодня я проверю твой проект.
<br> Дальнейшее общение будет происходить на "ты" если это не вызывает никаких проблем.
<br> Желательно реагировать на каждый мой комментарий ('исправил', 'не понятно как исправить ошибку', ...)
<br> Пожалуйста, не удаляй комментарии ревьюера, так как они повышают качество повторного ревью.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:

<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

### <font color='orange'>Общее впечатление</font>
* Большое спасибо за проделанную работу. Видно, что приложено много усилий.
* Спасибо, что структурируешь ноутбук. Приятно проверять такие работы.
* В этом проекте допущено несколько ошибок. Однако, их исправление не потребует много времени.
* Также я оставил несколько советов. Учти их, если останется время.
* Если будут вопросы, то можешь отправить проект с ними, я постараюсь ответить.
* Каждая ошибка — шанс научиться чему-то новому. Удачи в исправлениях!

### <font color='orange'>Общее впечатление (ревью 2)</font>
* Спасибо за исправления, работу приближается к идеальной.
* Однако, есть еще несколько недочетов, которые нужно исправить.
* Все новые замечания я пометил ключевым словом 'ревью 2'
* Если возникнут вопросы, то можешь смело их задать и отправить работу назад.
* Отправляй новую версию как только будут готовы изменения или появятся вопросы.

### <font color='orange'>Общее впечатление (ревью 3)</font>
* После исправлений работа значительно улучшилась.
* В ноутбуке немного нарушена структура (подробнее в совете помеченном 'ревью 3'). Однако, я понимаю, что это связано с несколькими итерациями исправлений.
* Просьба: в следующих раз перед отправкой проекта пробегаться по нему глазами и удалять лишние части.
* Также стоит измерять отдельно время обучения и время предсказания. В большинстве проектов время обучения играет намного меньшую роль. Один раз обучил и забыл, а применять надо чаще. Однако, бывает и наоборот.
* Проект зачтен. Удачи в дальнейшем обучении!

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

# 1. Подготовка данных

In [1]:
import os
import time
import pandas as pd
import numpy as np
import lightgbm as lgbm
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', 200)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:11,.2f}'.format
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

PATH = '/datasets/'
SEED = 21

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что все импорты собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

In [2]:
data = pd.read_csv('/datasets/autos.csv')

In [3]:
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [4]:
data = pd.read_csv(os.path.join(PATH, '/datasets/autos.csv'),
                   parse_dates=['DateCrawled', 'LastSeen', 'DateCreated'])
data.info()
data.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null datetime64[ns]
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null datetime64[ns]
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null datetime64[ns]
dtypes: datetime64[ns](3), int64(7), object(6)
memory usage: 43.3+ MB


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
182606,2016-03-25 09:57:16,3900,small,2008,manual,75,other,150000,3,gasoline,fiat,no,2016-03-25,0,49757,2016-04-06 09:46:20
324154,2016-04-07 05:36:23,1950,wagon,1998,auto,165,a4,150000,8,lpg,audi,no,2016-04-07,0,50969,2016-04-07 05:36:23
149264,2016-03-08 23:56:39,400,NaN,2016,manual,0,polo,150000,0,NaN,volkswagen,NaN,2016-03-08,0,42277,2016-03-12 14:44:12
291319,2016-04-04 20:43:08,650,sedan,1989,manual,54,golf,150000,2,gasoline,volkswagen,no,2016-04-04,0,98527,2016-04-04 20:43:08
286508,2016-03-20 23:37:22,4699,sedan,2005,manual,204,mondeo,150000,12,petrol,ford,no,2016-03-20,0,93128,2016-04-05 21:16:21


In [5]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,"354,369.00","354,369.00","354,369.00","354,369.00","354,369.00","354,369.00","354,369.00"
mean,"4,416.66","2,004.23",110.09,"128,211.17",5.71,0.00,"50,508.69"
std,"4,514.16",90.23,189.85,"37,905.34",3.73,0.00,"25,783.10"
min,0.00,"1,000.00",0.00,"5,000.00",0.00,0.00,"1,067.00"
25%,"1,050.00","1,999.00",69.00,"125,000.00",3.00,0.00,"30,165.00"
50%,"2,700.00","2,003.00",105.00,"150,000.00",6.00,0.00,"49,413.00"
75%,"6,400.00","2,008.00",143.00,"150,000.00",9.00,0.00,"71,083.00"
max,"20,000.00","9,999.00","20,000.00","150,000.00",12.00,0.00,"99,998.00"


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null datetime64[ns]
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null datetime64[ns]
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null datetime64[ns]
dtypes: datetime64[ns](3), int64(7), object(6)
memory usage: 43.3+ MB


#### Итак вот что бы видим:

- всего 354 369 строк
- в некоторых столбцах / features отсутствуют значения
- некоторые features являются категориальными  : VehicleType, NumberOfPictures, PostalCode, Gearbox, Model, FuelType, Brand, NotRepaired.
- Обратим внимание, что PostalCode и Model имеют высокую мощность, что может быть проблематично для древовидных алгоритмов.
- 3 features (DateCrawled, DateCreated, LastSeen) - это datetime, которые не могут быть обработаны моделями регрессии, поэтому мы разработаем еще одну features на ее основе.
 


<div class="alert alert-block alert-success">
<b>Успех:</b> Загрузка данных и первичный анализ сделаны хорошо.
</div>

 Удалим  столбцы


- NumberOfPictures - столбцы не заполнялись (всегда 0 или NaN)
- PostalCode - это номинальный категориальный признак. Существует более 8000 различных кодов, что делает функции слишком фрагментированными и безполезными.
- RegistrationMonth - не влияет на цену (только год)
 


In [7]:
data.drop(['NumberOfPictures', 'PostalCode', 'RegistrationMonth'], axis=1, inplace=True)

<div class="alert alert-block alert-success">
<b>Успех:</b> Молодец, что удалил эти признаки, они на самом деле бесполезны.
</div>

Удалим строки с Model = 'other' или Model = NaN - причина: цена сильно коррелирует с моделью

In [8]:
data = data.loc[data['Model'] != 'other']
data = data.loc[~data['Model'].isna()]

Удалим строки с низкой частотой (<20) model

In [9]:
lt_20 =data['Model'].value_counts().gt(19)
data = data.loc[data['Model'].isin(lt_20[lt_20].index)]

Удалим Price < 10

In [10]:
data = data.loc[data['Price'] >= 10]

Удалите выбросы в год регистрации <1960 (ошибка или антиквариат) и год регистрации> 2016 (данные за 2016 год ...)

In [11]:
data = data.loc[data['RegistrationYear'] >= 1960]
data = data.loc[data['RegistrationYear'] <= 2016]

Gearbox, VehicleType и FuelType - NaN будет заменено наиболее частым значением в модели.

In [12]:
data['Gearbox'].fillna(data.groupby('Model')['Gearbox'].transform(lambda x:x.value_counts().index[0]),inplace=True)
data['VehicleType'].fillna(data.groupby('Model')['VehicleType'].transform(lambda x:x.value_counts().index[0]),inplace=True)
data['FuelType'].fillna(data.groupby('Model')['FuelType'].transform(lambda x:x.value_counts().index[0]),inplace=True)

NotRepaired - заменим NaN на no - это, безусловно, наиболее частое значение

In [13]:
data['NotRepaired'].fillna('no',inplace=True)

In [14]:
data['Power'].quantile(0.995)

326.0

Power == 0 не существует (вероятно, не было загружено) и должно быть заменено наиболее частым значением для каждой модели. сделаем это для мощности <20

Также есть аномально большие значения (например, power = 20000). Мы также заменим значения в процентиле 99,5% (326).

In [15]:
data['Power'] = np.where(data['Power'] < 20 , data.groupby('Model')['Power'].transform(lambda x:x.value_counts().index[0]), data['Power'])
data['Power'] = np.where(data['Power'] > 326, data.groupby('Model')['Power'].transform(lambda x:x.value_counts().index[0]), data['Power'])

<div class="alert alert-block alert-success">
<b>Успех:</b> Удаление большого количества выбросов и ошибочных значений было проведено хорошо, молодец!
</div>

Создадим новую feature из feature datetime, прежде чем удалять их

In [16]:
data['days_seen'] = (data.LastSeen - data.DateCreated).dt.days
data.drop(['DateCrawled', 'DateCreated', 'LastSeen'], axis=1, inplace=True)

<div class="alert alert-block alert-success">
<b>Успех:</b> Молодец, что удалил еще и эти даты, они также не несут никакой полезной информации.
</div>

 
Назначим  dtype = category для категориальных feature  - это необходимо для работы моделей

In [17]:
categorical_features = data.select_dtypes(exclude=['number']).columns.tolist()
for c in categorical_features:
    data[c] = data[c].astype('category')

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично!
</div>

In [18]:
for cat in categorical_features:
    print(f'--- feature: {cat}, nuniques: {data[cat].nunique()} ---')
    print(data[cat].value_counts())

--- feature: VehicleType, nuniques: 8 ---
sedan          87359
small          75834
wagon          61039
bus            24738
convertible    17742
coupe          12240
suv             9038
other           1964
Name: VehicleType, dtype: int64
--- feature: Gearbox, nuniques: 2 ---
manual    233292
auto       56662
Name: Gearbox, dtype: int64
--- feature: Model, nuniques: 234 ---
golf       26660
3er        18599
polo       11987
corsa      11549
astra       9991
           ...  
delta         31
b_max         26
charade       26
9000          24
musa          22
Name: Model, Length: 234, dtype: int64
--- feature: FuelType, nuniques: 7 ---
petrol      195185
gasoline     89981
lpg           4074
cng            476
hybrid         123
other           90
electric        25
Name: FuelType, dtype: int64
--- feature: Brand, nuniques: 39 ---
volkswagen       67158
opel             33996
bmw              32937
mercedes_benz    26472
audi             25581
ford             21254
renault          1

<div class="alert alert-block alert-success">
<b>Успех:</b> В целом, этот шаг был проведен хорошо!
</div>

# 2. Обучение моделей

In [19]:
def split_groups (df, target_col):
    df_train, df_temp = train_test_split(df, test_size=0.4, random_state=12345)
    df_valid, df_test = train_test_split(df_temp, test_size=0.5, random_state=12345)

    features_train = df_train.drop([target_col], axis=1)
    target_train = df_train[target_col]
    features_valid = df_valid.drop([target_col], axis=1)
    target_valid = df_valid[target_col]
    features_test = df_test.drop([target_col], axis=1)
    target_test = df_test[target_col]
    return features_train, target_train, features_valid, target_valid, features_test, target_test

In [20]:
def print_rmse():
    rmse_train = mean_squared_error(target_train, prediction_train) ** 0.5
    rmse_valid = mean_squared_error(target_valid, prediction_valid) ** 0.5
    rmse_test = mean_squared_error(target_test, prediction_test) ** 0.5
    print(f'RMSE Train: {rmse_train:,.0f}, Valid: {rmse_valid:,.0f}, Test: {rmse_test:,.0f}, diff(Train,Valid):{100*(rmse_valid - rmse_train)/rmse_train:,.1f}%')

#### Random Forest

К сожалению, существует 234 уникальных модели. Для OHE потребуется 234-1 столбца, что сделает модель непрактичной (очень долгое время работы). Поэтому мы объединяем менее популярные модели в категорию «прочие».

In [21]:
data1 = data.copy()
top_50 = data1['Model'].value_counts()[:50].index
data1['Model'] = np.where(data1['Model'].isin(top_50),data1['Model'], 'other')
data_ohe = pd.get_dummies(data1, columns=categorical_features, drop_first=True)
features_train, target_train, features_valid, target_valid, features_test, target_test = split_groups(data_ohe, 'Price')

<div class="alert alert-block alert-success">
<b>Успех:</b> Это решением было очень интересным.
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Другим допустимым решением является удаление этого признака.
</div>

### <font color='orange'>  Переделал2  </font>

In [22]:
# создадим список категорийных признаков для градиентного бустинга CatBoost и LightGBM
cat_features = ['VehicleType', 'Gearbox', 'Model', 'FuelType','Brand', 'NotRepaired' ]

In [23]:
for i in cat_features:
    labelencoder = LabelEncoder()
    data[i] = labelencoder.fit_transform(data[i])

In [24]:
data.isnull().sum()

Price               0
VehicleType         0
RegistrationYear    0
Gearbox             0
Power               0
Model               0
Kilometer           0
FuelType            0
Brand               0
NotRepaired         0
days_seen           0
dtype: int64

In [25]:
target = data['Price']
features = data.drop('Price', axis=1)

In [26]:
features_train, features_ft, target_train, target_ft = train_test_split(    
    features, target, test_size=0.30, random_state=12345)

In [27]:
features_test, features_valid, target_test, target_valid = train_test_split(    
    features_ft, target_ft, test_size=0.5, random_state=12345)

In [37]:
%%time
# обучим основываясь на RandomForestRegressor
i = 0
result = pd.DataFrame(columns=['max_depth',  'n_estimators', 'rMSE'])
for n_estimators in range(10, 101, 10):
     for max_depth in range(1, 10, 2): 
        model_rfr = RandomForestRegressor(max_depth=max_depth, n_estimators=n_estimators, random_state=12345)
        model_rfr.fit(features_train, target_train)
        pred_valid_rfr = model_rfr.predict(features_valid)
        result.loc[i] = [max_depth, n_estimators, mean_squared_error(target_valid, pred_valid_rfr)**0.5]
        i += 1
result.sort_values('rMSE').head(5) 

CPU times: user 8min 44s, sys: 0 ns, total: 8min 44s
Wall time: 8min 45s


,max_depth,n_estimators,rMSE
39,9.00,80.00,"1,873.00"
44,9.00,90.00,"1,873.03"
24,9.00,50.00,"1,873.14"
49,9.00,100.00,"1,873.47"
19,9.00,40.00,"1,873.59"


In [31]:
%%time
# Обучим модель с оптимальными параметрами
model_cb = CatBoostRegressor(loss_function='RMSE', iterations=100 , depth=9, learning_rate=0.5)
model_cb.fit(features_train, target_train, cat_features=cat_features, verbose=False)
pred_valid_cb = model_cb.predict(features_test)
rmse = mean_squared_error(target_test, pred_valid_cb)**0.5
rmse

CPU times: user 36.7 s, sys: 3.98 s, total: 40.7 s
Wall time: 41.9 s


1546.824063984464

<!-- for depth in range(8,14):
    print(f'depth={depth}')
    model = RandomForestRegressor(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    prediction_train = model.predict(features_train)
    prediction_valid = model.predict(features_valid)
    prediction_test = model.predict(features_test)
    print_rmse() -->

<div class="alert alert-block alert-info">
<b>Совет (ревью 3): </b> Исходя из структуры проекта здесь должны быть обучена модель случайного леса.
</div>

Как видно выше, глубина = 9 обеспечивает приемлемый результат (глубина = 10 приводит к переобучению)

Мы снова разделились на 3 группы - это потому, что  dataframe  отличается, так как мы можем работать с категориальными данными в следующих 2 алгоритмах.

In [32]:
# for depth in range(8,14):
#     print(f'depth={depth}')
#     model = RandomForestRegressor(random_state=12345, max_depth=depth)
#     model.fit(features_train, target_train)
#     prediction_train = model.predict(features_train)
#     prediction_valid = model.predict(features_valid)
#     prediction_test = model.predict(features_test)
#     print_rmse()

<div class="alert alert-block alert-success">
<b> Сделал как ты сказал. так и есть переобучение </b>  
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Желательно было перебрать больше парамтров и в бОльшем диапазоне.
</div>

In [ ]:
# features_train, target_train, features_valid, target_valid, features_test, target_test = split_groups(data, 'Price')

<!--  features_train, target_train, features_valid, target_valid, features_test, target_test = split_groups(data, 'Price')  -->

#### CatBoost

<!-- %%time
for depth_ in range(10,16):
    model = CatBoostRegressor(loss_function="RMSE", depth=depth_,n_estimators=150)
    print (f'Depth: {depth_}')
    model.fit(features_valid, target_valid, cat_features=categorical_features, verbose=50)
    prediction_train = model.predict(features_train)
    prediction_valid = model.predict(features_valid)
    prediction_test = model.predict(features_test)
    print_rmse() -->

<div class="alert alert-block alert-success">
 Переделал 
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка (ревью 2):</b> Возможно я не совсем правильно выразился и поэтому ты не совсем правильно понял. У тебя есть 3 выборки:
    <br> 1. Обучающая. Всегда используется только для обучения модели.
    <br> 2. Валидационная. Используется для подбора параметров. То есть обучаем модель с разными параметрами на обучающей выборке и выбираем лучший параметр исходя из метрики на валидационной.
    <br> 3. Тестовая выборка используется для оценки качества модели в последнем шаге. Обучается модель на обучающей выборке используя только лучший параметр/параметры и измеряется качество на тестовой.
</div>

### <font color='orange'>  Переделал2  </font>

In [33]:
%%time
# обучим основываясь на библиотеке CatBoost
i = 0
result = pd.DataFrame(columns=['iteration','depth', 'learning_rate', 'rMSE'])
for iteration in range(20, 101, 10):
     for depth in range(1, 10, 2): 
        for learning_rate in np.arange(0.1, 1, 0.5):
            model = CatBoostRegressor(loss_function='RMSE', iterations=iteration , depth=depth, learning_rate=learning_rate)
            model.fit(features_train, target_train, cat_features=cat_features, verbose=False)
            pred_valid = model.predict(features_valid)
            result.loc[i] = [iteration, depth, learning_rate, mean_squared_error(target_valid, pred_valid)**0.5]
            i += 1
result.sort_values('rMSE').head(5)     

CPU times: user 21min 9s, sys: 2min 29s, total: 23min 38s
Wall time: 26min 27s


,iteration,depth,learning_rate,rMSE
89,100.00,9.00,0.60,"1,535.00"
79,90.00,9.00,0.60,"1,540.01"
69,80.00,9.00,0.60,"1,547.76"
59,70.00,9.00,0.60,"1,551.26"
87,100.00,7.00,0.60,"1,556.37"


In [38]:
%%time
# Обучим модель с оптимальными параметрами
model_cb = CatBoostRegressor(loss_function='RMSE', iterations=100 , depth=9, learning_rate=0.5)
model_cb.fit(features_train, target_train, cat_features=cat_features, verbose=False)
pred_valid_cb = model_cb.predict(features_test)
rmse = mean_squared_error(target_test, pred_valid_cb)**0.5
rmse

CPU times: user 38.3 s, sys: 2.33 s, total: 40.6 s
Wall time: 41.6 s


1546.824063984464

### <font color='orange'>  Переделал2  </font>

In [ ]:
# %%time
# for depth_ in range(12,16):
#     model = CatBoostRegressor(loss_function="RMSE", depth=depth_,n_estimators=150)
#     print (f'Depth: {depth_}')
#     model.fit(features_train, target_train, cat_features=categorical_features, verbose=50)
#     prediction_train = model.predict(features_train)
#     prediction_valid = model.predict(features_valid)
#     prediction_test = model.predict(features_test)
#     print_rmse()

#### LightGBM

Используя  num_leaves = 100 происходит переобучение  (RMSE train = 1384, RMSE valid = 1485 - разница 6,7%)

In [ ]:
# %%time
# nleaves = 30
# params = {
#  'boosting_type': 'gbdt',
#  'objective': 'regression',
#  'metric': {'root_mean_squared_error'},
#  'num_leaves': nleaves,
#  'learning_rate': 0.05,
#  'feature_fraction': 0.9,
#  'bagging_fraction': 0.8,
#  'bagging_freq': 5,
#  'verbose': 0
# }
# lgb_train = lgbm.Dataset(features_train, target_train)
# lgb_eval = lgbm.Dataset(features_valid, target_valid, reference=lgb_train)
# gbm = lgbm.train(params,
#                 lgb_train,
#                 num_boost_round=800,
#                 valid_sets=lgb_eval,
#                 early_stopping_rounds=5,verbose_eval=100)

# prediction_train = gbm.predict(features_train, num_iteration=gbm.best_iteration)
# prediction_valid = gbm.predict(features_valid, num_iteration=gbm.best_iteration)
# prediction_test = gbm.predict(features_test, num_iteration=gbm.best_iteration)
# print_rmse()

In [31]:
# %%time
# nleaves = 17
# params = {
#  'boosting_type': 'gbdt',
#  'objective': 'regression',
#  'metric': {'root_mean_squared_error'},
#  'num_leaves': nleaves,
#  'learning_rate': 0.05,
#  'feature_fraction': 0.9,
#  'bagging_fraction': 0.8,
#  'bagging_freq': 5,
#  'verbose': 0
# }
# lgb_train = lgbm.Dataset(features_train, target_train)
# lgb_eval = lgbm.Dataset(features_valid, target_valid, reference=lgb_train)
# gbm = lgbm.train(params,
#                 lgb_train,
#                 num_boost_round=800,
#                 valid_sets=lgb_eval,
#                 early_stopping_rounds=5,verbose_eval=100)

# prediction_train = gbm.predict(features_train, num_iteration=gbm.best_iteration)
# prediction_valid = gbm.predict(features_valid, num_iteration=gbm.best_iteration)
# prediction_test = gbm.predict(features_test, num_iteration=gbm.best_iteration)
# print_rmse()

Training until validation scores don't improve for 5 rounds
[100]	valid_0's rmse: 1645.11
[200]	valid_0's rmse: 1578.71
[300]	valid_0's rmse: 1556.82
Early stopping, best iteration is:
[330]	valid_0's rmse: 1553.36
RMSE Train: 1,504, Valid: 1,553, Test: 1,559, diff(Train,Valid):3.3%
CPU times: user 1min 57s, sys: 3.41 s, total: 2min 1s
Wall time: 2min 2s


<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что ты использовал внутренние методы модели для кодировки признаков.
</div>

In [34]:
%%time
# обучим основываясь на библиотеке lightGBM
i = 0
result = pd.DataFrame(columns=['max_depth','learning_rate', 'n_estimators', 'rMSE'])
for n_estimators in range(10, 101, 10):
     for max_depth in range(1, 10, 2): 
        for learning_rate in np.arange(0.1, 1, 0.2):
            model_lgb= lgb.LGBMRegressor(max_depth=max_depth, learning_rate=learning_rate, n_estimators=n_estimators)
            model_lgb.fit(features_train, target_train, categorical_feature=cat_features)
            pred_valid_lgb = model_lgb.predict(features_valid)
            result.loc[i] = [max_depth, learning_rate, n_estimators, mean_squared_error(target_valid, pred_valid_lgb)**0.5]
            i += 1
result.sort_values('rMSE').head(5)   

/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Brand', 'FuelType', 'Gearbox', 'Model', 'NotRepaired', 'VehicleType']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Brand', 'FuelType', 'Gearbox', 'Model', 'NotRepaired', 'VehicleType']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Brand', 'FuelType', 'Gearbox', 'Model', 'NotRepaired', 'VehicleType']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset

CPU times: user 15min 47s, sys: 4.63 s, total: 15min 52s
Wall time: 16min 2s


,max_depth,learning_rate,n_estimators,rMSE
241,7.00,0.30,100.00,"1,499.23"
216,7.00,0.30,90.00,"1,500.97"
246,9.00,0.30,100.00,"1,504.64"
191,7.00,0.30,80.00,"1,504.69"
221,9.00,0.30,90.00,"1,505.09"


In [35]:
%%time
# Обучим модель с оптимальными параметрами
model_lgb= lgb.LGBMRegressor(max_depth=7, learning_rate=0.3, n_estimators=100)
model_lgb.fit(features_train, target_train, categorical_feature=cat_features)
pred_valid_lgb = model_lgb.predict(features_test)
rmse = mean_squared_error(target_test, pred_valid_lgb)**0.5
rmse

/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Brand', 'FuelType', 'Gearbox', 'Model', 'NotRepaired', 'VehicleType']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


CPU times: user 6.56 s, sys: 40.8 ms, total: 6.6 s
Wall time: 6.6 s


1521.098515941413

LightGBM: теперь используем num_leaves = 31, без переобучения (RMSE train = 1426, RMSE valid = 1505 - diff 5%). Время выполнения: 48,7 с

#### обучим основываясь на линейной регрессии

In [39]:
%%time
model_LR = LinearRegression()
model_LR.fit(features_train, target_train)
pred_valid_LR = model_LR.predict(features_test)
rmse = mean_squared_error(target_test, pred_valid_LR)**0.5
rmse

CPU times: user 67 ms, sys: 21.8 ms, total: 88.8 ms
Wall time: 47.1 ms


2887.6565374256325

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Для чего в цикле каждый раз предсказываются значения для тестовой выборки?
    <br> Пожалуйста перебери больше параметров. Напомню, что это должно быть сделано на валидационной выборке.
    <br> Подбери в этом шаге лучшие параметры, а в следующем уже замерь время обучения каждой модели с лучшими параметрами и время предсказания. Например, время исполнения ячейки можно замерить с помощью вот так:
</div>

In [ ]:
%%time
print(123)

# 3. Анализ моделей

Как бы видим , LightGBM показывает лучшие результаты 

<div class="alert alert-block alert-danger">
<b>Ошибка(ревью 2):</b> Пожалуйста, обучи на это шаге модели с лучшими параметрами, измерь три показателя: время обучения, время предсказания, качество. Прими аргументированное решения, которые будет учитывать все три параметра.
    <br> Тестовая выборка должна использоваться только на этом шаге!
</div>

### Мною  были рассмотренны и обучены модели с помощью линейной регрессии, случайном лесе и библиотеках градиентного бустинга CatBoost и LightGBM. При обучении был произеден подбор оптимальных параметров, а затем обучены модели с параметрами показавшими наилучшие результаты.
### При этом были получены различные результаты как по качеству, так и по времени обучения моделей. Наибольшую скорость  показала линейная регрессии 47.1 ms, но при этом и наихудший результат по качеству RMCE= 2887,Случайный лес показал результат RMCE= 1547 при скорости 41.9 s. . При этом модель основаная на библиотеке CatBoost показала результат RMCE= 1546.37 - 41s временем обучения  , что немного превышает качество модели случайного леса . И наилучший результат показала модель основаная на библиотеке LightGBM RMCE= 1521.01 при скорости 5.98 с. 
### Данное сравнение результатов качества моделей и времеми выполнения операции позволяет рекомендовать для использования при построении модели градиентного бустинга, когда стоит ориентироватся на время выполнения операции библиотеку LightGBM, в случае когда необходима большая точность не смотря  на нельколько набольшее время выполнения следует использовать библиотеку CatBoost

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнена загрузка и подготовка данных
- [ ]  Выполнено обучение моделей
- [ ]  Есть анализ скорости работы и качества моделей